## Load merged data

In [2]:
from pathlib import Path
import json
import pandas as pd
mp = Path('.') / "merged" / "merged_data"
with mp.open("r", encoding="Latin1") as f:
    data = pd.read_json(f)

In [2]:
data.head()

,title,authors,doi,publication_date,abstract,repo_identifier,language,publisher,journal_name,journal_volume,journal_issue,issn,url,index
0,The Management of Grass Pastures for Brent Geese,"[J. A.Vickery, W. J.Sutherland, S. J.Lane]",10.2307/2404543,1994-5,An increasing number of brent geese now feed i...,crossref,EN,JSTOR,The Journal of Applied Ecology,31,2,0021-8901,http://dx.doi.org/10.2307/2404543,0
1,Control of Molinia caerulea on upland moors,"[R. H.Marrs, J. D. P.Phillips, P. A.Todd, J.Gh...",10.1111/j.0021-8901.2004.00901.x,2004-4,Summary 1 Molinia encroachment has been viewed...,crossref,EN,Wiley,Journal of Applied Ecology,41,2,0021-8901,http://dx.doi.org/10.1111/j.0021-8901.2004.009...,2
2,Long-distance relocation of nestboxes reduces ...,"[AlbertoSorace, FabrizioPetrassi, CarloConsiglio]",10.1080/00063650409461343,2004-7,None,crossref,EN,Informa UK Limited,None,51,2,0006-3657,http://dx.doi.org/10.1080/00063650409461343,3
3,Reducing the density of breeding gulls influen...,"[S. K.Finney, M. P.Harris, L. F.Keller, D. A.E...",10.1046/j.1365-2664.2003.00810.x,2003-6,Summary 1 By acting as both competitors and pr...,crossref,EN,Wiley,,40,3,0021-8901,http://dx.doi.org/10.1046/j.1365-2664.2003.008...,4
4,Elements that promote highway crossing structu...,"[WayneMcDonald, Colleen CassadySt Clair]",10.1111/j.1365-2664.2004.00877.x,2004-2-12,None,crossref,EN,Wiley,None,41,1,0021-8901,http://dx.doi.org/10.1111/j.1365-2664.2004.008...,5


In [ ]:
#Ellie
#Merge the pandas.Dataframe with index on the csv classification (Add Classification)
#Histogram of classifiers for which we have an abstract

# 5 abs      #10 abs
# Class 1 / Class2 /...
# Clean Abstracts with unicode.unicode
####################################################################################

# Each Journal --- If possible ISSUE + Volume granularity --- get negative articles
# Store in new JSON

# Journal Query ... Name, Vol, Issue (Daterange) -- Return all articles associated with that
# Markus: Openaire + Crossref, Implement Journal_Query
# Markus: CORE -- ISSN / Year / (Publication Title)

## Webscrape

In [1]:
b

In [3]:
no_abstract = data.loc[pd.isna(data['abstract'])]

In [4]:
def update_dict_w_abstract(row):
    try:
        row['abstract'] = unidecode.unidecode(get_abstract_from_doi(row['doi'])).strip()
    except Exception as e:
        row['abstract'] = None
    return row

In [127]:
acc = []

In [10]:
from concurrent.futures import as_completed, ThreadPoolExecutor
def multithreaded_fetch(data, start=0, end=None):
    start = start
    end = len(data) if end is None else end
    scrape = []
    accF = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        for ind, (_, row) in enumerate(data.iterrows()):
            if ind < start:
                continue
            if ind>end:
                break
            accF.append(executor.submit(update_dict_w_abstract, row))

        for future in as_completed(accF):
            try:
                datas = future.result()
                scrape.append(datas)
            except Exception as exc:
                print('generated an exception: %s' % (exc))
    return pd.DataFrame(scrape)

In [26]:
import time
import random
def slow_query(data, delay, start=0, end=0):
    start = start
    end = len(data) if end==0 else end
    acc=[]
    for ind, (_, row) in enumerate(data.iterrows()):
        if ind < start:
            continue
        if ind>end:
            break
        time.sleep(random.uniform(delay-0.25, delay+0.25))
        acc.append(update_dict_w_abstract(row))
    return pd.DataFrame(acc)

### Store scraping results

In [186]:
abstracts_scraped_2 = pd.DataFrame(acc)

In [193]:
abstracts_scraped_2.reset_index(drop=True, inplace=True)

In [200]:
dup_search = set(abstracts_scraped_2.columns)-set(["authors"])

In [198]:
abstracts_scraped_2.drop_duplicates(subset=dup_search, ignore_index=True, inplace=True)

In [199]:
path_scraped = Path('.') / "merged" / "scraped_data"
with path_scraped.open("w") as f:
    abstracts_scraped_2.to_json(f)

### Load scraping results

In [5]:
path_scraped = Path('.') / "merged" / "scraped_data"
with path_scraped.open("r") as f:
    abstracts_scraped_2= pd.read_json(f)

## Webscrape Remaining Data

In [6]:
left = abstracts_scraped_2.loc[pd.isna(abstracts_scraped_2['abstract'])]

In [7]:
left.groupby(['publisher']).count().sort_values(by="title", ascending=False).head(10)

,title,authors,doi,publication_date,abstract,repo_identifier,language,journal_name,journal_volume,journal_issue,issn,url,index
publisher,,,,,,,,,,,,,
Wiley,445,442,445,444,0,445,445,386,443,443,444,444,445
Oxford University Press (OUP),68,68,68,68,0,68,68,13,68,65,68,68,68
University of Wisconsin Press,68,68,68,67,0,68,68,52,67,67,67,67,68
JSTOR,38,38,38,38,0,38,38,35,38,35,38,38,38
Informa UK Limited,26,26,26,26,0,26,26,12,26,26,26,26,26
Elsevier BV,13,13,13,13,0,13,13,9,13,9,13,13,13
Universidad Nacional Autónoma de México (UNAM),9,9,0,9,0,9,9,9,9,9,9,9,9
Association of Applied Biologists,8,8,0,8,0,8,8,8,8,8,8,8,8
Springer Science and Business Media LLC,7,7,7,7,0,7,7,4,7,7,7,7,7


In [11]:
len(left)

813

### OXF Univ Press

In [27]:
oxf = left.loc[left["publisher"]=="Oxford University Press (OUP)"]

In [28]:
oxf

,title,authors,doi,publication_date,abstract,repo_identifier,language,publisher,journal_name,journal_volume,journal_issue,issn,url,index
123,Field Domiciles for Bumblebees,"[R. E.Fye, J. T.Medler]",10.1093/jee/47.4.672,1954-8-1,None,crossref,EN,Oxford University Press (OUP),Journal of Economic Entomology,47,4,1938-291X,http://dx.doi.org/10.1093/jee/47.4.672,642
134,"Flowers, Nectar and Insect Visits: Evaluating ...",[LCOMBA],10.1006/anbo.1998.0835,1999-4,None,crossref,EN,Oxford University Press (OUP),None,83,4,0305-7364,http://dx.doi.org/10.1006/anbo.1998.0835,702
153,Searching for a Manageable Pollinator for Acer...,"[ReislaOliveira, ClemensSchlindwein]",10.1603/029.102.0136,2009-2-1,None,crossref,EN,Oxford University Press (OUP),None,102,1,0022-0493,http://dx.doi.org/10.1603/029.102.0136,777
157,Comparison and Examination of Bombus occidenta...,"[RobinWhittington, Mark L.Winston]",10.1093/jee/97.4.1384,2004-8-1,None,crossref,EN,Oxford University Press (OUP),None,97,4,1938-291X,http://dx.doi.org/10.1093/jee/97.4.1384,805
161,Invasive Africanized honey bee impact on nativ...,"[DAVID W.ROUBIK, ROGELVILLANUEVA-GUTIÉRREZ]",10.1111/j.1095-8312.2009.01275.x,2009-8-25,None,crossref,EN,Oxford University Press (OUP),None,98,1,0024-4066,http://dx.doi.org/10.1111/j.1095-8312.2009.012...,820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,Adaptive use of nonlethal strategies for minim...,"[Suzanne A.Stone, Stewart W.Breck, JesseTimber...",10.1093/jmammal/gyw188,2017-2-2,None,crossref,EN,Oxford University Press (OUP),None,98,1,0022-2372,http://dx.doi.org/10.1093/jmammal/gyw188,18929
4632,Foraging behaviour and prey consumption by gre...,"[CecileVincent, VincentRidoux, Mike A.Fedak, B...",10.1093/icesjms/fsw102,2016-7-1,None,crossref,EN,Oxford University Press (OUP),None,73,10,1054-3139,http://dx.doi.org/10.1093/icesjms/fsw102,18984
4674,Effects of El Ni�o events on natural kelp beds...,[RGROVE],10.1006/jmsc.2002.1290,2002-10,None,crossref,EN,Oxford University Press (OUP),None,59,None,1054-3139,http://dx.doi.org/10.1006/jmsc.2002.1290,19156
4821,Drilling platforms as artificial reefs: distri...,[MPONTI],10.1006/jmsc.2002.1225,2002-10,None,crossref,EN,Oxford University Press (OUP),None,59,None,1054-3139,http://dx.doi.org/10.1006/jmsc.2002.1225,19771


In [29]:
result = slow_query(oxf, 3)

In [36]:
path_scraped = Path('.') / "merged" / "oxf_scraped_data"
with path_scraped.open("w") as f:
    result.to_json(f)

### University of Wisconsin Press

In [30]:
uwp = left.loc[left["publisher"]=="University of Wisconsin Press"]

NO SEARCHABLE ABSTRACTS

### Informa UK Limited

In [31]:
iul = left.loc[left["publisher"]=="Informa UK Limited"]

In [38]:
result = slow_query(iul, 2)

In [40]:
path_scraped = Path('.') / "merged" / "infuklim_scraped_data"
with path_scraped.open("w") as f:
    result.to_json(f)

### ELSEVIER

In [41]:
elsev = left.loc[left["publisher"]=="Elsevier BV"]

No useful Abstracts foound

## WILEY

In [8]:
wiley = left.loc[left["publisher"]=="Wiley"]
wiley.tail(20)

,title,authors,doi,publication_date,abstract,repo_identifier,language,publisher,journal_name,journal_volume,journal_issue,issn,url,index
4383,Managing tensions around urban flying-fox roosts,None,10.1111/aec.12738,2019-3-7,None,crossref,EN,Wiley,None,44,2,1442-9985,http://dx.doi.org/10.1111/aec.12738,18119
4388,RE-ESTABLISHING THE MUSK-OXOvibos moschatusIN ...,[Svend Andersen],10.1111/j.1748-1090.1966.tb01773.x,1966-1,None,openaire,EN,Wiley,International Zoo Yearbook,6,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1966.tb0...,18138
4427,Growth of pouch-young of the Red kangaroo Mega...,[MEREDITH J.CLARK],10.1111/j.1748-1090.1968.tb00451.x,1968-1,None,crossref,EN,Wiley,International Zoo Yearbook,8,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1968.tb0...,18277
4430,Towards grizzly bear population recovery in a ...,"[Nicholas C. Coops, Sean P. Kearney, Sean C. P...",10.1111/1365-2664.13259,2018-8-27,None,openaire,EN,Wiley,Journal of Applied Ecology,56,1,0021-8901,http://dx.doi.org/10.1111/1365-2664.13259,18286
4447,Last chance to see ?: ex situ conservation and...,[DavidDudgeon],10.1002/aqc.687,2005,None,crossref,EN,Wiley,None,15,2,1052-7613,http://dx.doi.org/10.1002/aqc.687,18340
4448,The reintroduction of Mountain gazelle Gazella...,"[KEVIN M.DUNHAM, TERENCE B.KICHENSIDE, NICKLIN...",10.1111/j.1748-1090.1993.tb03522.x,2007-12-18,None,crossref,EN,Wiley,International Zoo Yearbook,32,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1993.tb0...,18341
4478,Are wildlife escape ramps needed along Austral...,"[Ross L.Goldingay, Brendan D.Taylor, Jonathan ...",10.1111/emr.12319,2018-6-15,None,crossref,EN,Wiley,Ecological Management & Restoration,19,3,1442-7001,http://dx.doi.org/10.1111/emr.12319,18425
4480,Reintroduction of Scimitar-horned oryx Oryx da...,"[IAIN J.GORDON, J. PAUL.GILL]",10.1111/j.1748-1090.1993.tb03517.x,2007-12-18,None,crossref,EN,Wiley,International Zoo Yearbook,32,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1993.tb0...,18432
4484,The reintroduction programme for the Arabian o...,"[ARNAUDGRETH, GEORGSCHWEDE]",10.1111/j.1748-1090.1993.tb03518.x,2007-12-18,None,crossref,EN,Wiley,International Zoo Yearbook,32,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1993.tb0...,18447
4490,Reproduction of Cyprus mouflon Ovis gmelini op...,"[J. R. Bider, E. S. Hadjisterkotis]",10.1111/j.1748-1090.1992.tb02490.x,2007-12-18,None,openaire,EN,Wiley,International Zoo Yearbook,32,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1993.tb0...,18464


All data starting with 10.1111 

In [12]:
result = multithreaded_fetch(wiley)

In [23]:
result.loc[pd.isna(result['abstract'])]

,title,authors,doi,publication_date,abstract,repo_identifier,language,publisher,journal_name,journal_volume,journal_issue,issn,url,index
212,Breeding season responses of Skylarks Alauda a...,"[IAN G.HENDERSON, NIGEL R.CRITCHLEY, JONCOOPER...",10.1111/j.1474-919x.2001.tb04492.x,2001-4,None,crossref,EN,Wiley,Ibis,143,2,0019-1019,http://dx.doi.org/10.1111/j.1474-919x.2001.tb0...,1274
351,FIRST SUCCESSFUL HANDREARING OF AN ABANDONED B...,[David B.Wingate],10.1111/j.1474-919x.1972.tb02593.x,2008-4-3,None,crossref,EN,Wiley,Ibis,114,1,0019-1019,http://dx.doi.org/10.1111/j.1474-919x.1972.tb0...,1998
82,Alloparental care and kleptoparasitism in the ...,"[Beatriz E.Arroyo, Jesus T.García]",10.1046/j.1474-919x.2002.00083.x,2002-9-13,None,crossref,EN,Wiley,None,144,4,0019-1019,http://dx.doi.org/10.1046/j.1474-919x.2002.000...,458
335,ADVANCEMENT OF LAYING OF GREAT TITS BY THE PRO...,[HansKällander],10.1111/j.1474-919x.1974.tb00133.x,2008-4-3,None,crossref,EN,Wiley,None,116,3,0019-1019,http://dx.doi.org/10.1111/j.1474-919x.1974.tb0...,1926
330,Sex-related local recruitment in colonial and ...,"[LAJOSSASVÁRI, ZOLTÁNHEGYI]",10.1111/j.1474-919x.2000.tb07691.x,2008-6-28,None,crossref,EN,Wiley,None,142,1,0019-1019,http://dx.doi.org/10.1111/j.1474-919x.2000.tb0...,1899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,Shrubland resilience varies across soil types:...,"[Carissa L.Wonkka, DiracTwidwell, Jason B.West...",10.1890/15-0066.1,2015-5-11,None,crossref,EN,Wiley,None,None,None,1051-0761,http://dx.doi.org/10.1890/15-0066.1,20681
4990,Amounts of Big Sagebrush in Plant Communities ...,[Harold W. Cooper],10.2307/1930318,None,None,openaire,EN,Wiley,Ecology,None,None,None,None,20656
4484,The reintroduction programme for the Arabian o...,"[ARNAUDGRETH, GEORGSCHWEDE]",10.1111/j.1748-1090.1993.tb03518.x,2007-12-18,None,crossref,EN,Wiley,International Zoo Yearbook,32,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1993.tb0...,18447
4585,The reintroduction of the Arabian oryx: into Oman,[M. R. Stanley Price],10.1111/j.1748-1090.1985.tb02536.x,1986-1,None,openaire,EN,Wiley,International Zoo Yearbook,24,1,0074-9664,http://dx.doi.org/10.1111/j.1748-1090.1985.tb0...,18799


In [22]:
path_scraped = Path('.') / "merged" / "wiley_scraped_data"
with path_scraped.open("w") as f:
    result.to_json(f)

## Experiment with doi api

In [18]:
doi = ("10.1111/emr.12319")

In [19]:
get_abstract_from_doi(doi)

'Road escape ramps are structures developed in the USA to enable large mammals that become trapped on the roadside of a wildlife fence to escape the road and avoid vehicle collision. They are now commonly installed in eastern Australia to enable the Koala (Phascolarctos cinereus) and other fauna to escape through a roadside exclusion fence and return to the forest. We investigated the use of seven of 14 escape ramps over three years on the Oxley Highway at Port Macquarie in New South Wales. The Swamp Wallaby (Wallabia bicolor) was the most frequently detected species, traversing the escape ramps on 502 occasions, followed by bandicoots (Isoodon macrourus and Perameles nasuta) on 148 occasions. Various other species were detected but no Koalas. Swamp Wallabies moved through the escape ramps in the reverse direction (i.e. towards the road) in 53% of detections of that species and bandicoots in 14% of their detections. There was no obvious pattern by these species of proportionately highe

In [160]:
    scraper = cloudscraper.create_scraper()
    doi_data = scraper.get(f"https://www.doi.org/{doi}", timeout=10)
    doi_bs = BeautifulSoup(doi_data.text)

In [125]:
pattern = re.compile(r"\n\s*Abstract\s*(.*)\n", flags=re.IGNORECASE)
result = pattern.search(doi_bs.body.get_text())

In [161]:
doi_bs.body.get_text()#result.group(0)

"\n\n\nSkip to Article Content\nSkip to Article Information\n\n\n\n\n\n\n\n\n\n\n\n\nWorking off-campus? Learn about our remote access options\n\n\n\n\n\n\n\nAccess byCOCHRANE GRACING\n\n\n\n\n\n\n\n\n\nAccess byCOCHRANE GRACING\n\n\n\nSearch withinThis JournalZSL PublicationsWiley Online LibrarySearch term  Advanced Search  Citation SearchSearch term  Advanced Search  Citation SearchSearch term  Advanced Search  Citation Search\n\n\n\n\n\n\n\n\n\n\nLogin / Register\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJournals\n\n\n\nJournal of Zoology\nAnimal Conservation\nInternational Zoo Yearbook\nRemote Sensing in Ecology and Conservation\n\n\nOpen access\n\n\n\n\n\n\n\n\n\nzsl.org\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nInternational Zoo YearbookVolume 24, Issue 1 p. 56-65\n\n\n\nAmphibians: review of zoo breeding programmes\n\nEDWARD J. MARUSKA\nDirector, Zoological Society of Cincinnati, 3400 Vine Street, Cincinnati, Ohio 45220, USASearch for more papers by this author\n